In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
import mylib as ml

# 파일 경로 설정 (로컬 경로로 수정)
data = pd.read_csv('./000660.csv')
data = ml.dataRename2(data)
data

c:\big18\dl-dev\dl-dev\project\mylib.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['날짜','시간']] = data[['날짜', '시간']].astype(str)
c:\big18\dl-dev\dl-dev\project\mylib.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['날짜'].astype(str) + data['시간'].astype(str)
c:\big18\dl-dev\dl-dev\project\mylib.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,close,date
0,90000,2022-09-20 09:11:00
1,90200,2022-09-20 09:12:00
2,90300,2022-09-20 09:13:00
3,90300,2022-09-20 09:14:00
4,90300,2022-09-20 09:15:00
...,...,...
17385,172900,2024-08-02 15:16:00
17386,172900,2024-08-02 15:17:00
17387,172700,2024-08-02 15:18:00
17388,173000,2024-08-02 15:19:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
import mylib as ml

# 파일 경로 설정 (로컬 경로로 수정)
data = pd.read_csv('./000660.csv')
data=data[['날짜','시간','종가']]
data[['날짜','시간']] = data[['날짜', '시간']].astype(str)
data['date'] = data['날짜'].astype(str) + data['시간'].astype(str)
data['시간'] = data['시간'].str.zfill(4)
data['date'] = pd.to_datetime(data['date'], format='%Y%m%d%H%M')
data = data.drop(['날짜'],axis=1)
data = data.drop(['시간'],axis=1)
data.rename(columns={'종가':'SP500','date':'ds'}, inplace=True)
nov_dir = data
nov_train = nov_dir[:int(len(nov_dir) * 0.8)]
nov_test = nov_dir[int(len(nov_dir) * 0.8):]

def preprocess_data(data_path):
    """
    CSV 데이터를 읽어 시계열 분석을 위한 전처리를 수행한 후 DataFrame을 반환하는 함수
    Args:
        data_path (str): 데이터를 포함한 CSV 파일 경로
    Returns:
        pd.DataFrame: 시계열 분석을 위한 전처리된 DataFrame
    """
    df = data_path
    # try:
    #     # 'Date' 컬럼을 날짜형으로 변환
    #     df['ds'] = pd.to_datetime(df['Date'])
    # except pd.errors.OutOfBoundsDatetime:
    #     print("Error: Date format might be invalid. Please check your data.")
    #     return None
    # 'SP500' 컬럼의 값을 스케일링하여 'y' 컬럼에 저장
    df['y'] = preprocessing.scale(df['SP500'])
    # 'unique_id' 컬럼을 추가하여 각 데이터셋을 식별 가능하게 함
    df['unique_id'] = 1.0
    # 불필요한 컬럼('Date', 'SP500')을 제거
    df.drop(columns=['SP500'], inplace=True)
    # 컬럼 순서를 재정렬
    df = df.reindex(columns=['unique_id', 'ds', 'y'])
    return df

# 훈련 데이터와 테스트 데이터를 전처리하여 각각 DataFrame으로 저장
df_train = preprocess_data(nov_train)
df_test = preprocess_data(nov_test)
df_train.info()  # 훈련 데이터의 정보 출력

def add_predict_column(df, model, input_size):
    """
    df_test에 예측 결과를 추가하여 반환하는 함수
    Args:
        df: 테스트 데이터의 DataFrame
        model: 예측에 사용할 모델
        input_size: 모델이 입력으로 사용할 데이터 크기
    Returns:
        예측 결과가 추가된 DataFrame
    """
    df_predict = pd.DataFrame()
    
    for i in range(len(df) - input_size):
        # 입력 크기에 맞게 슬라이싱된 데이터를 사용해 예측
        arr = df.iloc[i:i+input_size].copy()
        predict = model.predict(df=arr, verbose=True)
        
        # predict가 DataFrame인지 확인하고, 아니라면 변환
        if isinstance(predict, pd.DataFrame):
            df_predict = pd.concat([df_predict, predict], ignore_index=True)
        else:
            df_predict = pd.concat([df_predict, pd.DataFrame(predict)], ignore_index=True)
    
    return df_predict

input_size = 100  # TimesNet의 입력 크기 설정
network = [TimesNet(h=1, input_size=input_size, max_steps=100)]  # TimesNet 모델 구성
model = NeuralForecast(models=network, freq='D')  # NeuralForecast 객체 생성
model.fit(df=df_train)  # TimesNet 모델을 사용하여 훈련 수행

# 테스트 데이터에 예측 결과 추가
df_predict = add_predict_column(df_test, model, input_size)

# 실제 값과 예측 값을 비교하는 그래프 생성
plt.figure(figsize=(15, 5))
plt.plot(df_test['ds'], df_test['y'], label="Actual")  # 실제 주가 데이터
plt.plot(df_predict['ds'], df_predict['TimesNet'], label="Predictions")  # 예측한 주가 데이터
plt.title("Actual vs. Predicted")  # 그래프 제목 설정
plt.legend()  # 범례 추가
plt.show()  # 그래프 출력


c:\big18\dl-dev\dl-dev\project\mylib.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['날짜','시간']] = data[['날짜', '시간']].astype(str)
c:\big18\dl-dev\dl-dev\project\mylib.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['날짜'].astype(str) + data['시간'].astype(str)
c:\big18\dl-dev\dl-dev\project\mylib.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13912 entries, 0 to 13911
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   unique_id  13912 non-null  float64       
 1   ds         13912 non-null  datetime64[ns]
 2   y          13912 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 326.2 KB


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates